In [1]:
import tensorflow as tf

In [2]:
def read_csv_format_with_image(filename_queue, width, height, channels=3):
    reader = tf.TextLineReader()
    key, line = reader.read(filename_queue)
    _ = tf.decode_csv(line, record_defaults=[[''], [0]])
    label = _[1]# tf.cast(_[1], tf.int32)
    filepath = _[0]
    
    bytestream = tf.read_file(filepath)
    image = tf.image.decode_png(bytestream, channels=channels)
    
    #TODO: add preprocessing
    
    image = tf.image.resize_images(image, height, width)

    label = tf.cast(label, tf.int32)
    image = tf.cast(image, tf.float32)
    return image, label

In [3]:
def input_pipeline(filenames, width, height, batch_size, channels=3, read_threads=8, num_epochs=None):
    filename_queue = tf.train.string_input_producer(filenames, num_epochs=num_epochs, shuffle=True)
    example_list = [read_csv_format_with_image(filename_queue, width, height, channels) for _ in range(read_threads)]
    min_after_dequeue = 1
    capacity = min_after_dequeue + 3 * batch_size
    image_batch, label_batch = tf.train.shuffle_batch_join(
        example_list, batch_size=batch_size, capacity=capacity, min_after_dequeue=min_after_dequeue)
    return image_batch, label_batch

In [ ]:
image, label = input_pipeline(['datas/ordered.txt'], 32, 32, 4)

with tf.Session() as sess:
    sess.run( tf.initialize_all_variables() )
    
    # start input enqueue threads
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        i = 0
        while not coord.should_stop():
            i += 1
            if i > 10:
                break;
            batch = sess.run([image, label])
            print batch[1]
    except tf.error.outOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()

    # wait for threads to finish
    coord.join(threads)

In [ ]:
image, label = input_pipeline(['datas/label0.txt', 'datas/label1.txt', 'datas/label2.txt', 'datas/label3.txt'], 32, 32, 4)

with tf.Session() as sess:
    sess.run( tf.initialize_all_variables() )
    
    # start input enqueue threads
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        i = 0
        while not coord.should_stop():
            i += 1
            if i > 10:
                break;
            batch = sess.run([image, label])
            print batch[1]
    except tf.error.outOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()

    # wait for threads to finish
    coord.join(threads)